# Step 1. Follow along with the tutorial to gain an understanding of the process

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

Loading the dataset returns four NumPy arrays:

The train_images and train_labels arrays are the training set, the data which is the model uses to learn.
The model is tested against the test set, the test_images, and test_labels arrays.
The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. The labels are an array of integers, ranging from 0 to 9. These correspond to the class of number the image represents:

```
Label	Class
0	0
1	1
2	2
3	3
4	4
5	5
6	6
7	7
8	8
9	9
```

Each image is mapped to a single label. Since the class names are not included with the dataset, store them here to use later when plotting the images:

# Step 2. In a new .ipynb notebook, reproduce the results utilizing the "QMNIST" dataset 

For data Transforming we are perfoming two things together.
- transforms.ToTensor() : Converting into tensors, tensor will map these values to the range [0, 1]
- transforms.Normalize() : Normalizing the data, Normalization shifts and scales them to the range [-1, 1], first tuple (0.5,) is  mean, and the second tuple (0.5,) is Sd.

In [2]:
# Loading the  dataset and Data Transforming
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.QMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.QMNIST(root='./data', train=False, download=True, transform=transform)


100%|██████████| 9704059/9704059 [00:00<00:00, 34992682.36it/s]


Extracting ./data/QMNIST/raw/qmnist-train-images-idx3-ubyte.gz to ./data/QMNIST/raw


100%|██████████| 463024/463024 [00:00<00:00, 19567192.42it/s]

Extracting ./data/QMNIST/raw/qmnist-train-labels-idx2-int.gz to ./data/QMNIST/raw


100%|██████████| 9742279/9742279 [00:00<00:00, 35813770.67it/s]


Extracting ./data/QMNIST/raw/qmnist-test-images-idx3-ubyte.gz to ./data/QMNIST/raw


100%|██████████| 526800/526800 [00:00<00:00, 22158523.68it/s]

Extracting ./data/QMNIST/raw/qmnist-test-labels-idx2-int.gz to ./data/QMNIST/raw


In [3]:
# Creating Data Loader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Step 3. Report on the results in terms of prediction accuracy on the train and test datasets

# Step 4. Choose one of the proposed modifications below:

Add another Dense layer of 128 nodes
Increase the current number of nodes in the layer to 256
Hypothesize how it would change the performance results

# Step 5. Modify the model based on the chosen method and train

# Step 6. Report on the results of the modified model and if it matches your hypothesis

# Step 7. Experiment with different optimizers, loss functions, dropout, and activation functions, and observe the change in performance as you tune these hyperparameters.